In [151]:
import pandas as pd
import spacy
from fuzzywuzzy import fuzz


nlp = spacy.load("en_core_web_sm")
nlp.add_pipe(("sentencizer"), before="parser")


def filter_dataframe(input_text,df):

    doc = nlp(input_text)

    column_name = None
    operator = None
    value = None

    for token in doc:
        print(f"{token.pos_}")
        print(f"{token.text}")
        # Columns math
        if token.pos_ in ('NOUN','PROPN'):
            for col in df.columns:
                score = fuzz.ratio(token.text.lower(), col.lower())
                if score >= 60:
                    column_name = col
                    break
        
        # Mapping
        elif token.pos_ in ['VERB', 'ADJ','X','ADP']:
            if token.text.lower() in ['greater', 'more']:
                operator = '>'
            elif token.text.lower() in ['less', 'fewer']:
                operator = '<'
            elif token.text.lower() in ['equal', 'equals', 'same','=']:
                operator = '=='
            elif token.text in ['>', '<', '==']:
                operator = token.text
        
       
        elif operator and (token.pos_ in ['NUM', 'NOUN', 'PROPN']):
            value = token.text
            if token.pos_ == 'NUM':
                if column_name in df.select_dtypes(include=['number']).columns:
                    value = float(value)
                else:
                    value = str(value)

    print(f"Operator: {operator}")
    print(f"{column_name},{operator},{value}")
        
    if column_name and operator and value:
        if column_name in df.select_dtypes(include=['number']).columns:
            value = float(value)
        else:
            value = str(value)

        filtered_df = df.query(f"{column_name} {operator} @value")
        return filtered_df
    
    else:
        return "Not Found"
